## Imports

In [1]:
import os
import pandas as pd
import numpy as np
import logging

from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from sklearn.model_selection import train_test_split

## Get the workspace

In [2]:
subscription_id = '95bcf3b7-9903-4d62-9b7b-00484a87a6cb'
resource_group = 'ResearchProject'
workspace_name = 'AutoML'

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded.")
except:
    print("Workspace not accessible.")

Workspace configuration succeeded.


## Get compute

In [3]:
# Choose a name for your CPU cluster
cpu_cluster_name = "Automl-Compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_F2S_V2', max_nodes=3)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Load Data

In [4]:
df = pd.read_csv('../Data/mnist.csv')

df.rename(columns={'5': 'label'}, inplace=True)

df = df.sample(frac=1).reset_index(drop=True)

train_data = df[18000:]
test_data = df[:18000]

In [5]:
df.head(10)

,label,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.608,0.609,0.610,0.611,0.612,0.613,0.614,0.615,0.616,0.617
0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Data source and format (Pandas (local) or TabularDataset (remote compute))
if not os.path.isdir('data'):
    os.mkdir('data')
    
# Save the train data to a csv to be uploaded to the datastore
pd.DataFrame(train_data).to_csv("data/train_data.csv", index=False)
pd.DataFrame(test_data).to_csv("data/test_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='mnist', overwrite=True, show_progress=True)

# Upload the training data as a tabular dataset for access during training on remote compute
train_dataset = Dataset.Tabular.from_delimited_files(path=ds.path('mnist/train_data.csv'))
test_dataset = Dataset.Tabular.from_delimited_files(path=ds.path('mnist/test_data.csv'))

label = "label"

Uploading an estimated of 2 files
Uploading ./data\test_data.csv
Uploaded ./data\test_data.csv, 1 files out of an estimated total of 2
Uploading ./data\train_data.csv
Uploaded ./data\train_data.csv, 2 files out of an estimated total of 2
Uploaded 2 files


## Configure Experiment

In [10]:
# Configure experiment settings
# Data featurization (automatically scaled and normalized)
# Exit criteria

automl_settings = {
    "experiment_timeout_hours" : 1.5,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "n_cross_validations": 2,
    "primary_metric": 'accuracy',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             experiment_exit_score = 1,
                             enable_onnx_compatible_models=True,
                             training_data = train_dataset,
                             label_column_name = label,
                             **automl_settings
                            )

# Choose a name for experiment
experiment_name = 'Mnist_AutoML'
experiment = Experiment(ws, experiment_name)

In [11]:
remote_run = experiment.submit(automl_config, show_output = False)

print(remote_run)

remote_run.wait_for_completion()

best_run_customized, fitted_model_customized = remote_run.get_output()

Running on remote.
Run(Experiment: Mnist_AutoML,
Id: AutoML_879e615f-b029-4b73-84b8-af8405364c60,
Type: automl,
Status: NotStarted)
